In [116]:
import numpy as np
from sklearn.linear_model import LinearRegression
import io
import seaborn as sns
import pandas as pd
import os
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge

In [137]:
X = pd.read_csv('data/cox2/cox2_matrix_alphabet_NNdbr_UpTo3chains.csv', sep = ',')
Y = pd.read_csv('data/cox2/cox2_y.csv', sep = ',', usecols=[0])
header =X.columns
X = X.to_numpy()
Y = Y.to_numpy()

yl = list()
for i in range(Y.shape[0]):
    yl.append(Y[i][0])

Data = np.column_stack((X, Y[:, 0])) 
data_train, data_test= train_test_split(Data, test_size = 0.5, random_state = 3)

train_x = data_train[:, 0:274]
train_y = data_train[:, 274]
test_x = data_test[:, 0:274]
test_y = data_test[:, 274]
data = X
y = np.array(yl) 

In [138]:
def R_rise_2 (Eps, y, y_mean):
    result1 = np.sum(Eps**2)
    result2 = np.sum((y-y.mean())**2)
    return 1 - result1 / result2


def Correlation(X, Y):
    X_mean = X.sum() / X.shape[0]
    Y_mean = Y.sum() / Y.shape[0]
    res = ((X) * (Y)).sum()
    res1 = (((X) ** 2)) 
    res2 = (((Y) ** 2)) 
    
    return  res / (((res1.sum() * res2.sum())) ** 0.5)

In [217]:
size_buf = 15
num_selections = 15
min_correlation = 0.7
#100
#15

In [218]:
model = Ridge(alpha=0.2)

In [219]:
n = data.shape[0]
m = data.shape[1]

In [220]:
y_mean = data.sum(axis = 1)
y_mean = y_mean / m
#y_mean

In [221]:
buf_r = np.array([0.0 for i in range(size_buf)])
buf_size_ar = np.array([0 for i in range(num_selections)])
iter_buf = 0

buf_numbers = np.array([[-1 for i in range(size_buf)] for k in range(num_selections)])
buf_numbers_1 = np.array([[-1 for i in range(size_buf)] for k in range(num_selections)])

buf = np.array([[[float(i) for i in range(size_buf)] for j in range(n)] for k in range(num_selections)])

X1 = np.array([[float(i) for i in range(1)] for j in range(n)])
X2 = np.array([[float(i) for i in range(2)] for j in range(n)])
buf.shape

(15, 467, 15)

In [222]:
%%time
for i in range(m):
    X1[:,0] = data[:,i]
    model.fit(X1, y)
    if iter_buf < size_buf:
        tmp = model.predict(X1)
        ar = np.array([])

        for j in range (iter_buf):
            t = Correlation (buf[0, :,j], tmp)
            ar = np.append (ar, t)
        #print(ar)
        if (iter_buf == 0) or (ar.max() < min_correlation):    
            buf_r[iter_buf] = R_rise_2(y - tmp, y, y_mean)
            buf_numbers[0, iter_buf] = i
            buf[0, :, iter_buf] = tmp[:]
            iter_buf += 1
            
    else:
        tmp = model.predict(X1)
        ind = buf_r.argmin()
        ar = np.array([])
        for j in range (buf.shape[2]):
            t = Correlation (buf[0, :,j], tmp)
            ar = np.append (ar, t)
        #print(ar)
        if (ar.max() < min_correlation) and (buf_r[ind] < R_rise_2(y - tmp, y, y_mean)): 
            buf_r[ind] = R_rise_2(y - tmp, y, y_mean)
            buf_numbers[0, ind] = i
            buf[0, :, ind] = tmp[:]
buf_numbers_1[0, :] = -2
print(buf_numbers_1, end="\n\n\n\n")
buf_size_ar[0] = iter_buf
print(buf_size_ar[0])

[[-2 -2 -2 -2 -2 -2 -2 -2 -2 -2 -2 -2 -2 -2 -2]
 [-1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1]]



1
CPU times: user 228 ms, sys: 0 ns, total: 228 ms
Wall time: 227 ms


In [223]:
%%time
for k in range (1, num_selections):
    iter_buf = 0
    for i in range(buf_size_ar[k - 1]):
        for j in range(0, m):
            #print(data[:,i].shape)
            X2[:, 0] = buf[k - 1, :, i]
            X2[:, 1] = data[:, j]
            model.fit(X2, y)
            if iter_buf < size_buf:
                tmp = model.predict(X2)
                ar = np.array([])
            
                for j1 in range (iter_buf):
                    t = Correlation (buf[0, :,j1], tmp)
                    ar = np.append (ar, t)
                #print(j, ar)
                
                if (iter_buf == 0) or (ar.max() < min_correlation):
                    buf_r[iter_buf] = R_rise_2(y - tmp, y, y_mean)
                    buf_numbers[k, iter_buf] = j
                    #print("*******",i, j)
                    buf_numbers_1[k, iter_buf] = i
                    buf[k, :,iter_buf] = tmp[:]
                    iter_buf += 1

            else:
                tmp = model.predict(X2)
                ind = buf_r.argmin()
                ar = np.array([])
                for j1 in range (buf.shape[2]):
                    t = Correlation (buf[0, :,j1], tmp)
                    ar = np.append (ar, t)
                if (ar.max() < min_correlation) and (buf_r[ind] < R_rise_2(y - tmp, y, y_mean)):
                    buf_r[ind] = R_rise_2(y - tmp, y, y_mean)
                    buf_numbers[k, ind] = j
                    buf_numbers_1[k, ind] = i
                    buf[k, :, ind] = tmp[:]
    buf_size_ar[k] = iter_buf

    print("selection", k + 1, "from", num_selections)

selection 2 from 15
selection 3 from 15
selection 4 from 15
selection 5 from 15
selection 6 from 15
selection 7 from 15
selection 8 from 15
selection 9 from 15
selection 10 from 15
selection 11 from 15
selection 12 from 15
selection 13 from 15
selection 14 from 15
selection 15 from 15
CPU times: user 6.02 s, sys: 184 ms, total: 6.2 s
Wall time: 3.11 s


In [224]:
buf_numbers

array([[ 0, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1],
       [ 0, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1],
       [ 0, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1],
       [ 0, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1],
       [ 0, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1],
       [ 0, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1],
       [ 0, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1],
       [ 0, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1],
       [ 0, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1],
       [ 0, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1],
       [ 0, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1],
       [ 0, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1],
       [ 0, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1],
       [ 0, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1],
       [ 0, -1, -1, -1, -1, -1, -1

In [225]:
buf_numbers_1

array([[-2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2],
       [ 0, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1],
       [ 0, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1],
       [ 0, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1],
       [ 0, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1],
       [ 0, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1],
       [ 0, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1],
       [ 0, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1],
       [ 0, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1],
       [ 0, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1],
       [ 0, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1],
       [ 0, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1],
       [ 0, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1],
       [ 0, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1],
       [ 0, -1, -1, -1, -1, -1, -1

In [226]:
buf_size_ar

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [227]:
lis = []
for i in range(buf_size_ar[num_selections - 1]):
    lis.append([buf_numbers[num_selections - 1][i]])

In [228]:
for i in range(num_selections - 1, 0, -1):
    #print(i)
    for j in range(buf_size_ar[i - 1]):
        lis[j].append(buf_numbers[i - 1][buf_numbers_1[i,j]])

In [229]:
lis

[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]

In [230]:
for i in range(buf_size_ar[num_selections-1]):
    lis[i].sort()
    print(lis[i])

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [231]:
clf = Ridge(alpha=0.1)
X_train, X_test, y_train, y_test = train_test_split(data, y, train_size=0.6, test_size=0.4)
clf.fit(X_train, y_train)
clf.score(X_test, y_test)

-0.5743732248523916

In [232]:
for i in range(buf_size_ar[num_selections-1]):
    clf.fit(X_train[:,np.unique(lis[i])], y_train)
    print(clf.score(X_test[:,np.unique(lis[i])], y_test))

0.004907942674701515


In [233]:
clf_Lin = LinearRegression()
clf_MGUA = LinearRegression()
res_MGUA = clf_MGUA.fit(X_train[:,np.unique(lis[2])], y_train)
res_Lin = clf_Lin.fit(X_train, y_train)

IndexError: list index out of range

In [175]:
predict_MGUA = clf_MGUA.predict(X_test[:,np.unique(lis[2])])

In [176]:
predict_Lin = clf_Lin.predict(X_test)

In [3]:
for i in range(predict_Lin.shape[0]):
    print(predict_MGUA[i], '&', predict_Lin[i], '&', y_test[i])

NameError: name 'predict_Lin' is not defined

In [178]:
mean_err = 0
mean_sq_err = 0

In [179]:
mean_err = (predict_MGUA - y_test).sum() / predict_MGUA.shape[0]
mean_err = abs(mean_err)

In [180]:
mean_sq_err = ((predict_MGUA - y_test)**2).sum() / predict_MGUA.shape[0]


In [181]:
print("Средняя ошибка", mean_err)
print("Среднеквадратическая ошибка", mean_sq_err)

Средняя ошибка 3.095265991950349
Среднеквадратическая ошибка 1098.657654437439
